# hakank-TSP 

This uses [hakank's code on GitHub](https://github.com/hakank/hakank/blob/master/minizinc/tsp.mzn). That example uses 16 nodes in a 0-1 variable integer programming model. 

I am solving for a more reasonable 48. So I am not optimistic about this being an improvement over Bologna.

In [130]:
 %load_ext iminizinc

The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [131]:
import pandas as pd
from geopy.distance import great_circle

In [132]:
cities = pd.read_csv("data/city.csv")

In [133]:
cdict = cities.set_index('City').T.to_dict('list')

In [134]:
# Test: distance from Montgomery, AL to Tallahassee, FL
int(great_circle(tuple(cdict['Montgomery']), tuple(cdict['Tallahassee'])).miles)

177

In [144]:
psize = 10 # size of problem (number of cities used)
keys = list(cdict.keys())
keys = keys[:psize]

# Here are values for the variables/parameters shared with MiniZinc:
n = len(keys)
num_edges = n*(n-1)
targets = list(range(n))        
E = []
for i in range(n):
    tar = targets.copy()
    del tar[tar.index(i)]
    for j in range(len(tar)):
        E.append([i+1,tar[j]+1])       
c = []
for i in range(num_edges):
    c.append(int(great_circle(tuple(cdict[keys[E[i][0]-1]]), 
                              tuple(cdict[keys[E[i][1]-1]])).miles))

for i in range(num_edges):
    print(keys[E[i][0]-1], ' to ', keys[E[i][1]-1], ' = ', c[i])

Montgomery  to  Phoenix  =  1494
Montgomery  to  Little Rock  =  385
Montgomery  to  Sacramento  =  2013
Montgomery  to  Denver  =  1159
Montgomery  to  Hartford  =  990
Montgomery  to  Dover  =  763
Montgomery  to  Tallahassee  =  177
Montgomery  to  Atlanta  =  145
Montgomery  to  Boise  =  1793
Phoenix  to  Montgomery  =  1494
Phoenix  to  Little Rock  =  1131
Phoenix  to  Sacramento  =  632
Phoenix  to  Denver  =  585
Phoenix  to  Hartford  =  2210
Phoenix  to  Dover  =  2058
Phoenix  to  Tallahassee  =  1638
Phoenix  to  Atlanta  =  1588
Phoenix  to  Boise  =  737
Little Rock  to  Montgomery  =  385
Little Rock  to  Phoenix  =  1131
Little Rock  to  Sacramento  =  1629
Little Rock  to  Denver  =  776
Little Rock  to  Hartford  =  1168
Little Rock  to  Dover  =  975
Little Rock  to  Tallahassee  =  554
Little Rock  to  Atlanta  =  458
Little Rock  to  Boise  =  1412
Sacramento  to  Montgomery  =  2013
Sacramento  to  Phoenix  =  632
Sacramento  to  Little Rock  =  1629
Sacramento  

In [145]:
%%minizinc -m bind

% number of nodes 
int: n;

% set of arcs 
int: num_edges;
array[1..num_edges, 1..2] of 1..n: E;

% distance from node i to node j 
array[1..num_edges] of int: c;

% x[i,j] = 1 means that the salesman goes from node i to node j 
array[1..num_edges] of var 0..1: x;

% y[i,j] is the number of cars, which the salesman has after leaving
% node i and before entering node j; in terms of the network analysis,
% y[i,j] is a flow through arc (i,j) 
array[1..num_edges] of var int: y;

% the objective is to make the path length as small as possible 
var int: total = sum(i in 1..num_edges) (c[i] * x[i]);
solve :: int_search(
    [x[i] | i in 1..num_edges] ++
    [y[i] | i in 1..num_edges] ++
    [total],
   first_fail, % "occurrence",
   indomain_max,
   complete
   )
   minimize total;

constraint
   % the salesman leaves each node i exactly once 
   forall(i in 1..n) (
        sum(k in 1..num_edges where E[k,1] = i) (x[k]) = 1)
   /\
   % the salesman enters each node j exactly once 
   forall(j in 1..n) (
        sum(k in 1..num_edges where E[k,2] = j) (x[k]) = 1)
   /\
   % Constraints above are not sufficient to describe valid tours, so we
   % need to add constraints to eliminate subtours, i.e. tours which have
   % disconnected components. Although there are many known ways to do
   % that, I invented yet another way. The general idea is the following.
   % Let the salesman sells, say, cars, starting the travel from node 1,
   % where he has n cars. If we require the salesman to sell exactly one
   % car in each node, he will need to go through all nodes to satisfy
   % this requirement, thus, all subtours will be eliminated. 
   % 

   % if arc (i,j) does not belong to the salesman's tour, its capacity
   % must be zero; it is obvious that on leaving a node, it is sufficient
   % to have not more than n-1 cars 
    forall(k in 1..num_edges) (
      y[k] >= 0
      /\
      y[k] <= (n-1) * x[k])
   /\
   % node[i] is a conservation constraint for node i 
   forall(i in 1..n) (
      % summary flow into node i through all ingoing arcs 
      (
      sum(k in 1..num_edges where E[k,2] = i) (y[k])
      % plus n cars which the salesman has at starting node 
      + (if i = 1 then n else 0 endif)
      )
      = % must be equal to 
      % summary flow from node i through all outgoing arcs 
      (
      sum(k in 1..num_edges where E[k,1] = i) (y[k])
      % plus one car which the salesman sells at node i 
      + 1));

In [146]:
print('x = ',x,'\ny = ',y)

x =  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] 
y =  [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]


In [147]:
trav = []
for i in range(num_edges):
    if x[i] == 1: 
        trav.append(E[i])

In [148]:
[trav, keys]

[[[1, 3],
  [2, 4],
  [3, 2],
  [4, 10],
  [5, 6],
  [6, 7],
  [7, 9],
  [8, 1],
  [9, 8],
  [10, 5]],
 ['Montgomery',
  'Phoenix',
  'Little Rock',
  'Sacramento',
  'Denver',
  'Hartford',
  'Dover',
  'Tallahassee',
  'Atlanta',
  'Boise']]

In [149]:
tour = [keys[edge[0][0]-1]]

In [150]:
next = trav[0][1]
for i in range(1,psize-1):
    for j in range(psize):
        if trav[j][0] == next:
            tour.append(keys[trav[j][0]-1])
            next = trav[j][1] 
print(tour)

['Montgomery', 'Little Rock', 'Phoenix', 'Sacramento', 'Boise', 'Denver', 'Hartford', 'Dover', 'Atlanta', 'Tallahassee', 'Montgomery', 'Little Rock', 'Phoenix', 'Sacramento', 'Boise', 'Denver', 'Hartford', 'Dover', 'Atlanta', 'Tallahassee']


## Bibliography

* [GitHub site for TSP solved for 48 US States.](https://github.com/toddwschneider/shiny-salesman) But the data is in binary. 
* [Here is longitudes and latitudes](http://www.xfront.com/us_states/), then use [this](https://pypi.org/project/geopy/) to calculate the distances. 